# Finding exact states
Most of the time we talk about states like $|J = 2, m_J = 0, I_{tot} = 0 \rangle$, but these are not exact eigenstates of the Hamiltonian due to the spin rotation mixing (primarily the $c_1$-term). Here finding the exact forms of given states in a particular electric field.

## Import packages

In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
plt.style.use(['seaborn-darkgrid','thesis'])
import numpy as np
from scipy.linalg import eigh

from centrex_TlF_hamiltonian.states import (State, CoupledBasisState, UncoupledBasisState, generate_uncoupled_states_ground, 
                                            find_state_idx_from_state, find_closest_vector_idx, ElectronicState, find_exact_states,
                                            reorder_evecs)
from centrex_TlF_hamiltonian.hamiltonian import generate_uncoupled_hamiltonian_X, generate_uncoupled_hamiltonian_X_function, XConstants
from centrex_TlF_hamiltonian.hamiltonian.utils import matrix_to_states

## Diagonalizing Hamiltonian
We can also calculate the shifts in the energies of states by diagonalizing the Hamiltonian

Start by defining the X-state Hamiltonian

In [2]:
# Generate quantum numbers
QN_X_uc = generate_uncoupled_states_ground(range(0,6))

# Generate Hamiltonian as function of E and B
H_X_EB = generate_uncoupled_hamiltonian_X_function(generate_uncoupled_hamiltonian_X(QN_X_uc))

## Function for finding states of interest

In [3]:
def find_exact_states(approx_states, H, QN):
    """
    Finds the exact state that corresponds to the given approx_state
    """
    D, V = np.linalg.eigh(H)
    indices = []
    for approx_state in approx_states:
        indices.append(find_state_idx_from_state(H, approx_state, QN))
    
    states = matrix_to_states(V[:,indices], QN)
    
    return states

## Define states
Define the states we're interested in

In [4]:
J0_singlet = (1*UncoupledBasisState(J=0, mJ=0, I1=1/2, m1=1/2, I2=1/2, m2=-1/2,Omega=0, P=+1, electronic_state=ElectronicState.X)
              - 1*UncoupledBasisState(J=0, mJ=0, I1=1/2, m1=-1/2, I2=1/2, m2=1/2,Omega=0, P=+1, electronic_state=ElectronicState.X))/np.sqrt(2)

J1_singlet = (1*UncoupledBasisState(J=1, mJ=0, I1=1/2, m1=1/2, I2=1/2, m2=-1/2,Omega=0, P=-1, electronic_state=ElectronicState.X)
              - 1*UncoupledBasisState(J=1, mJ=0, I1=1/2, m1=-1/2, I2=1/2, m2=1/2,Omega=0, P=-1, electronic_state=ElectronicState.X))/np.sqrt(2)

J2_0pp = 1*UncoupledBasisState(J=2, mJ=0, I1=1/2, m1=1/2, I2=1/2, m2=1/2,Omega=0, P=+1, electronic_state=ElectronicState.X)
J2_0mm = 1*UncoupledBasisState(J=2, mJ=0, I1=1/2, m1=-1/2, I2=1/2, m2=-1/2,Omega=0, P=+1, electronic_state=ElectronicState.X)
J2_singlet = (1*UncoupledBasisState(J=2, mJ=0, I1=1/2, m1=1/2, I2=1/2, m2=-1/2,Omega=0, P=+1, electronic_state=ElectronicState.X)
              - 1*UncoupledBasisState(J=2, mJ=0, I1=1/2, m1=-1/2, I2=1/2, m2=1/2,Omega=0, P=+1, electronic_state=ElectronicState.X))/np.sqrt(2)
J2_triplet_0 = (1*UncoupledBasisState(J=2, mJ=0, I1=1/2, m1=1/2, I2=1/2, m2=-1/2,Omega=0, P=+1, electronic_state=ElectronicState.X)
               + 1*UncoupledBasisState(J=2, mJ=0, I1=1/2, m1=-1/2, I2=1/2, m2=1/2,Omega=0, P=+1, electronic_state=ElectronicState.X))/np.sqrt(2)

In [5]:
approx_states = [J0_singlet, J1_singlet, J2_singlet]

## Find exact states

In [6]:
B = np.array([0,0, 1e-6])
E = 100
H = H_X_EB(np.array([0,0,E]), B)

In [7]:
exact_states = find_exact_states(approx_states, H, QN_X_uc)

In [18]:
for approx_state, exact_state in zip(approx_states, exact_states):
    print(f"Exact state corresponding to \n|approx>= \n{approx_state.state_string(digits=3)}\n is \n|exact>= \n{exact_state.state_string(digits=3)}")

Exact state corresponding to 
|approx>= 
0.707 x |X, J = 0, mJ = 0, I₁ = 1/2, m₁ = 1/2, I₂ = 1/2, m₂ = -1/2, P = +, Ω = 0>
-0.707 x |X, J = 0, mJ = 0, I₁ = 1/2, m₁ = -1/2, I₂ = 1/2, m₂ = 1/2, P = +, Ω = 0>
 is 
|exact>= 
0.707-0.000j x |X, J = 0, mJ = 0, I₁ = 1/2, m₁ = 1/2, I₂ = 1/2, m₂ = -1/2, P = +, Ω = 0>
-0.707+0.000j x |X, J = 0, mJ = 0, I₁ = 1/2, m₁ = -1/2, I₂ = 1/2, m₂ = 1/2, P = +, Ω = 0>

Exact state corresponding to 
|approx>= 
0.707 x |X, J = 1, mJ = 0, I₁ = 1/2, m₁ = 1/2, I₂ = 1/2, m₂ = -1/2, P = -, Ω = 0>
-0.707 x |X, J = 1, mJ = 0, I₁ = 1/2, m₁ = -1/2, I₂ = 1/2, m₂ = 1/2, P = -, Ω = 0>
 is 
|exact>= 
0.705-0.000j x |X, J = 1, mJ = 0, I₁ = 1/2, m₁ = 1/2, I₂ = 1/2, m₂ = -1/2, P = -, Ω = 0>
-0.705+0.000j x |X, J = 1, mJ = 0, I₁ = 1/2, m₁ = -1/2, I₂ = 1/2, m₂ = 1/2, P = -, Ω = 0>
0.049-0.000j x |X, J = 1, mJ = 1, I₁ = 1/2, m₁ = -1/2, I₂ = 1/2, m₂ = -1/2, P = -, Ω = 0>
-0.049+0.000j x |X, J = 1, mJ = -1, I₁ = 1/2, m₁ = 1/2, I₂ = 1/2, m₂ = 1/2, P = -, Ω = 0>

Exact state corres